## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---
## First, I'll import the necessary libraries

In [14]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
%matplotlib qt

## Function to compute the camera calibration using chessboard images

In [15]:
# Calibrates the camera based on checkerboard images 
def calibrate_camera():
    # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
    objp = np.zeros((6*9,3), np.float32)
    objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

    # Arrays to store object points and image points from all the images.
    objpoints = [] # 3d points in real world space
    imgpoints = [] # 2d points in image plane.

    # Make a list of calibration images
    images = glob.glob('camera_cal/calibration*.jpg')

    # Step through the list and search for chessboard corners
    for fname in images:

        # Read the image and convert to greyscale
        img = plt.imread(fname)
        gray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)

        # Find the chessboard corners
        ret, corners = cv2.findChessboardCorners(gray, (9,6),None)

        # If found, add object points, image points
        if ret == True:
            objpoints.append(objp)
            imgpoints.append(corners)

            # Draw and display the corners
            #img = cv2.drawChessboardCorners(img, (9,6), corners, ret)
            #plt.imshow('img',img)
            #cv2.waitKey(500)

    # Update the vales used in calibrateCamera
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
    return ret, mtx, dist, rvecs, tvecs

# Test the calibration results 
#img = plt.imread('camera_cal/calibration1.jpg') # bring in distorted image 
#dst_test = cv2.undistort(img,mtx,dist,None,mtx)
#plt.title("Undistorted")
#plt.imsave("undistorted_checkerboard.jpg", dst_test)



## Function that applies the distortion correction to the images 

In [42]:
# Takes in image, camera matrix, and distortion coefficients and outputs undistorted image
def undistort(img, mtx, dist):
    return cv2.undistort(img, mtx, dist, None, mtx)

## Function that create threshold binary images

In [45]:
# Pipeline for creating binary image modified from class notes 
def get_binary(img, s_thresh=(160,230), sx_thresh=(35,100)):
    img = np.copy(img)
    # Convert to HLS color space
    hls = cv2. cvtColor(img, cv2.COLOR_RGB2HLS)
    L = hls[:,:,1]
    S = hls[:,:,2]
    
    # Sobel in the x direction
    sobelx = np.absolute(cv2.Sobel(L, cv2.CV_64F, 1, 0))
    scaled_sobel = np.uint8(255*sobelx/np.max(sobelx))
    
    # Apply sobelx threshold 
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 1
    
    # Apply an S threshold
    thresh = (150, 255)
    sbinary = np.zeros_like(S)
    sbinary[(S > thresh[0]) & (S < thresh[1])] = 1
    
    # Combine each channel 
    #binary = np.dstack((np.zeros_like(sxbinary), sxbinary, sbinary)) * 255  # See what each channel does
    binary = np.zeros_like(sbinary)
    binary[(sbinary == 1) | (sxbinary == 1)] = 1
    return binary


## Functions to apply Perspective Transform for Birds Eye View and reverse

In [47]:
# Applies the perspective transform on the images 
def perspective_trans(img):
    # Get information about the image 
    yvert = 455    # This is where the lines stop in the middle of the image 
    ymax = img.shape[0]
    xmax = img.shape[1]
    img_size = (img.shape[1], img.shape[0])
    
    # Get the transformation vertices 
    src = np.float32([[200,ymax],[590, yvert], [700, yvert], 
                          [1150,ymax]], dtype=np.int32)
    dst = np.float32([[300,ymax],[300,0],[950,0],[950,ymax]])
    
    # Calculate perspective transform matrix 
    M = cv2.getPerspectiveTransform(src,dst)
    warped = cv2.warpPerspective(img, M, img_size, dst)
    
    return warped, M

# Reverses the perspective transform back to original image 
def reverse_perspective(img):
    # Get information about the image 
    yvert = 455    # This is where the lines stop in the middle of the image 
    ymax = img.shape[0]
    xmax = img.shape[1]
    img_size = (img.shape[1], img.shape[0])
    
    # Get the transformation vertices (reverse of perspective_trans)
    src = np.float32([[300,ymax],[300,0],[950,0],[950,ymax]])
    dst = np.float32([[200,ymax],[590, yvert], [700, yvert], 
                          [1150,ymax]], dtype=np.int32)
    
    # Calculate perspective transform matrix 
    M = cv2.getPerspectiveTransform(src,dst)
    warped = cv2.warpPerspective(img, M, img_size, dst)
    
    return warped, M


## Function to detect lane pixels and fit to find the lane boundary.

In [39]:
def find_lane_pixels(binary_warped):
    # Get histogram from bottom half of the image 
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    out_image = np.dstack((binary_warped, binary_warped, binary_warped))
    
    # Get the starting points for left and right halves 
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint
    
    
    # Set hyperparameters for sliding windows 
    nwindows = 12  # Number of windows 
    margin = 40  # Width of windows +/- margin 
    minpix = 50   # Number of pixels to recenter
    window_height = np.int(binary_warped.shape[0]//nwindows)
    
    # Find positions of nonzero pixels
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    # Set initial positions 
    leftx_current = leftx_base
    rightx_current = rightx_base
    
    # Go through each window and store lane pixel indices 
    left_lane_inds = []
    right_lane_inds = []
    for window in range(nwindows):
        # Find boundaries for window
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        
        # Draw the windows 
        #cv2.rectangle(out_image,(win_xleft_low, win_y_low), (win_xleft_high, win_y_high), (0,255,0), 2)
        #cv2.rectangle(out_image,(win_xright_low, win_y_low), (win_xright_high, win_y_high), (0,255,0), 2)
        
        # Get nonzero pixels inside window and add to list
        left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) &
                    (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
        right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) &
                     (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
        left_lane_inds.append(left_inds)
        right_lane_inds.append(right_inds)
        
        # Recenter window if necessary 
        if len(left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[left_inds]))
        if len(right_inds) > minpix:
            rightx_current = np.int(np.mean(nonzerox[right_inds]))
            
    # Concatenate the arrays of indices 
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)
    
    # Get left and right line pixel positions 
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds]
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    
    return leftx, lefty, rightx, righty, out_image

# Use for images to fit the polynomial
def fit_first_polynomial(binary_warped):
    # Convert from pixel space to meters 
    ym_per_pix = 30/720   # meters/pixel in y direction
    xm_per_pix = 3.7/700  # meters/pixel in x direction 
    
    # Get the lane pixels 
    leftx, lefty, rightx, righty, out_image = find_lane_pixels(binary_warped)
    
    # Fit a second order polynomial (in pixel space)
    left_poly = np.polyfit(lefty, leftx, 2)
    right_poly = np.polyfit(righty, rightx, 2)
    
    # Fit a second order polynomial (in real space)
    left_poly_real = np.polyfit(lefty*ym_per_pix, leftx*xm_per_pix, 2)
    right_poly_real = np.polyfit(righty*ym_per_pix, rightx*xm_per_pix, 2)
    
    # Plot lane boundaries 
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0])
    #left_fitx = left_poly[0]*ploty**2 + left_poly[1]*ploty + left_poly[2]
    #right_fitx = right_poly[0]*ploty**2 + right_poly[1]*ploty + right_poly[2]
    
    
    # Use ploty to draw the polygon (green lane) 
    left_evals = left_poly[0]*(ploty**2) + left_poly[1]*ploty + left_poly[2]
    right_evals = right_poly[0]*(ploty**2) + right_poly[1]*ploty + right_poly[2]
    image_array = []
    ind = len(left_evals)
    for i in range(1,ind):
        image_array.append([int(left_evals[ind-i]), int(ploty[ind-i])])
    for i in range(1,ind):
        image_array.append([int(right_evals[i]), int(ploty[i])])
    polygon_array = [image_array]
    polygon_array = np.asarray(polygon_array, dtype=np.int32)
    cv2.fillPoly(out_image, polygon_array, color=[0,255,0])
    
    
    # Color in the lane lines 
    out_image[lefty, leftx] = [255,0,0]
    out_image[righty, rightx] = [0, 0, 255]
    #plt.plot(left_fitx, ploty, color='yellow')
    #plt.plot(right_fitx, ploty, color='yellow')
    
    return left_poly_real, right_poly_real, ploty, out_image


    
    
    

## Determine the curvature of the lane and vehicle position with respect to center

In [20]:
def get_curvature(left_poly, right_poly, ploty):
    # Convert from pixel space to meters 
    ym_per_pix = 30/720   # meters/pixel in y direction
    xm_per_pix = 3.7/700  # meters/pixel in x direction 
    
    # Measure curvature from bottom of image 
    yeval = np.max(ploty)*ym_per_pix
    
    # Calculation for radius of curvature 
    left_curve_rad = ((1 + (2*left_poly[0]*yeval + left_poly[1])**2)**(1.5)) / (np.absolute(2*left_poly[0]))
    right_curve_rad = ((1 + (2*right_poly[0]*yeval + right_poly[1])**2)**(1.5)) / (np.absolute(2*right_poly[0]))
    
    # Calculate distance from center 
    left = left_poly[0]*(yeval)**2 + left_poly[1]*yeval + left_poly[2]
    right = right_poly[0]*(yeval)**2 + right_poly[1]*yeval + right_poly[2]
    center = 640 * xm_per_pix
    loc = (left + right) / 2.0 
    dist_center = center - loc
    
    return left_curve_rad, right_curve_rad, dist_center

## Warp the detected lane boundaries back onto the original image.

In [21]:
def draw_lanes(image, binary_image):
    # Fit the line and get curvature and distance
    left_fit, right_fit, ploty, out_image = fit_first_polynomial(binary_image)
    left_curve, right_curve, dist_center = get_curvature(left_fit, right_fit, ploty)
    
    # Reverse the perspective transform 
    lanes, M = reverse_perspective(out_image)
    
    # Create the image 
    overlay = cv2.addWeighted(image, 1.0, lanes, 0.35, 0)
    cv2.putText(overlay, "Radius of Curvature = {}m".format(int((left_curve + right_curve)/2)),
                (30, 75), cv2.FONT_HERSHEY_PLAIN, 4.0, (255, 255, 255), 3)
    cv2.putText(overlay, "Distance from Center = {0:.2f}m".format(round(dist_center,2)),
                (30, 135), cv2.FONT_HERSHEY_PLAIN, 4.0, (255, 255, 255), 3)
    
    return overlay
    '''
    plt.imshow(image)
    plt.imshow(lanes, cmap='jet', alpha=0.35)
    plt.text(30,75, ('Radius of Curvature: %dm' %((left_curve + right_curve) / 2)), fontsize=16, color='white')
    plt.text(30,135, ('Distance from center: %.2fm' %(dist_center)), fontsize=16, color='white')
    plt.savefig(save_file)
    plt.clf()
    plt.close()
    '''

## Pipeline for processing image

In [27]:
# Takes in image and assumes camera matrix/distortion coefficients have been created 
def process_image(image):
    # Undistort the image
    undist = undistort(image,mtx,dist)
    
    # Transform the image to top down view and get binary
    transformed, M = perspective_trans(undist)
    binary = get_binary(transformed)
    
    # Overlay lane lines
    return draw_lanes(undist, binary)

## Test the functions on test_images

In [40]:
# Get camera calibration values 
ret, mtx, dist, rvecs, tvecs = calibrate_camera()

# Undistort and save all of the test images in an array (test_images)
test_images = []
images = glob.glob('test_images/*.jpg')
for fname in images:
    img = plt.imread(fname)
    test_images.append(process_image(img))
    
# Save the test images
for i in range(len(test_images)):
    filename = ('output_images/output' + str(i) + '.jpg')
    plt.imsave(filename, test_images[i])

# Pipline for video

In [24]:
from moviepy.editor import VideoFileClip
#from IPython.display import HTML

In [48]:
# Get camera calibration values 
ret, mtx, dist, rvecs, tvecs = calibrate_camera()

output_file = 'output1video.mp4'
clip1 = VideoFileClip('project_video.mp4')
output = clip1.fl_image(process_image)
%time output.write_videofile(filename=output_file, audio=False)


[MoviePy] >>>> Building video output1video.mp4
[MoviePy] Writing video output1video.mp4


100%|█████████▉| 1260/1261 [01:11<00:00, 18.50it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: output1video.mp4 

CPU times: user 11min 28s, sys: 11.3 s, total: 11min 39s
Wall time: 1min 11s
